<a href="https://colab.research.google.com/github/born99/video_summarize_dsf/blob/master/Copy_of_Demo7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd
import os
os.chdir('/content/drive/My Drive/video_summarize_dsf')

!pip install chainer
!pip install torch
!pip install scikit-video

/content


In [3]:
import numpy as np
import os
import json

from func.nets import vid_enc, vid_enc_vgg19
from chainer import serializers

import sys
sys.path.append('script/')
from summarize import get_flabel
from func.sampling.vsum import VSUM

import chainer
from chainer import configuration

import torch

/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda111) version 9.4.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda111>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))


In [4]:
# settings
feat_type = 'smt_feat' # smt_feat (proposed) or vgg

# load embedding model
if feat_type == 'smt_feat':
    model = vid_enc.Model()
    print('model:', type(model))
    serializers.load_npz('data/trained_model/model_par', model)
    print('model:', type(model))
elif feat_type == 'vgg':
    model = vid_enc_vgg19.Model()
    print('model:', type(model))
else:
    raise RuntimeError('[invalid feat_type] use smt_feat or vgg')

model: <class 'func.nets.vid_enc.Model'>
model: <class 'func.nets.vid_enc.Model'>


In [5]:
# prepair output dir
d_name = 'summe'
dataset_root = 'data/{}/'.format(d_name)
out_dir = 'results/{:}/{:}/'.format(d_name, feat_type)
print('save to: ', out_dir)

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

save to:  results/summe/smt_feat/


In [6]:
# load dataset metadata
dataset = json.load(open(dataset_root + 'dataset.json'))
video_id = [d['videoID'] for d in dataset]

print('Video list:')
for vi in video_id:
    print('-', vi)

Video list:
- Cooking


In [7]:
# summarize video
v_id = 'Cooking'

with configuration.using_config('train', False):
    with chainer.no_backprop_mode():
        vsum = VSUM(v_id, model, dataset=d_name, seg_l=5)

_, frames, _ = vsum.summarizeRep(seg_l=5, weights=[1.0, 0.0])

# get 0/1 label for each frame
fps = vsum.dataset.data['fps']
fnum = vsum.dataset.data['fnum']
label = get_flabel(frames, fnum, fps, seg_l=5)

ロードした動画のID: Cooking
budget: 2
feat: <class 'numpy.ndarray'>


In [8]:
!pwd
!python script/summarize.py
!python script/evaluate.py results/summe/smt_feat

/content/drive/My Drive/video_summarize_dsf
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda111) version 9.4.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda111>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))
resultの保存先: results/summe/smt_feat/
Traceback (most recent call last):
  File "script/summarize.py", line 54, in <module>
    torch.load_npz('./data/trained_model/model_par', model)
AttributeError: module 'torch' has no attribute 'load_npz'
Traceback (most recent call last):
  File "script/evaluate.py", line 93, in <module>
    res 

In [9]:
# write summarized video
from skvideo.io import vread, vwrite

# video_path = 'video_summarize_dsf/data/summe/%s.webm' % v_id
video_path = 'data/summe/video.mp4'

video_data = vread(video_path)
sum_vid = video_data[label.ravel().astype(np.bool), :,:,:]

print ('writing video to', 'sum_%s.mp4'%v_id)
vwrite('sum_%s.mp4'%v_id, sum_vid)

FileNotFoundError: ignored